<a href="https://www.kaggle.com/code/nazaninmahmoudy/dogs-and-cats-classification-transfer-learning?scriptVersionId=253268313" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# 🐾  Cat vs Dog Classifier With Transfer Learning (2025)

#### 📌 **Project Overview**

This project focuses on binary image classification using deep learning and **transfer learning** techniques. The objective is to classify images as either Cat or Dog using a pre-trained MobileNetV2 model. The **goal** is to demonstrate how transfer learning can achieve **high accuracy** even with a limited dataset, while also integrating essential computer vision techniques like image preprocessing and evaluation metrics.


#### 🖼️ **Dataset**

The dataset contains 10,000 labeled images, evenly distributed between cats and dogs. The images are preprocessed with resizing and normalizationto improve generalization. The data was split into training (4000), validation (1000), and test (5000) sets.



#### **⚙️ Model & Approach**

The model architecture is based on MobileNetV2 from TensorFlow Keras applications, pre-trained on ImageNet. This base model was frozen during training, and custom dense layers were added on top for binary classification.

🔧 **Model Components**:

✅ Pretrained MobileNetV2 (feature extractor)

✅ GlobalAveragePooling2D

✅ Dense layer with ReLU activation

✅ Dropout (to reduce overfitting)

✅ Dense output layer with softmax activation (2 classes)

📦 Loss Function: sparse_categorical_crossentropy                   
🚀 Optimizer: Adam                       
🎯 Metrics: Accuracy                 

#### **📊 Dataset**

The dataset consists of a total of 25,000 labeled images of cats and dogs:

train/cats: 10,000 images

train/dogs: 10,000 images

test/cats: 2,500 images

test/dogs: 2,500 images

In this project, a subset of 5,000 images (2,500 per class) from the test folder was used as the final test set.
The training data (20,000 images total) was further split into:

Training set: 4,000 images (stratified sample)

Validation set: 1,000 images

All images were resized to **224x224**, normalized, and augmented during training to improve generalization.



#### 📈 Evaluation Metrics

The model achieved outstanding performance across all standard classification metrics. When evaluated on the official test set consisting of 5,000 images (2,500 cats and 2,500 dogs), it reached an overall accuracy of 97%, with balanced precision, recall, and F1-scores of 0.97 for both classes. 
Furthermore, to assess its generalization ability, the model was also tested on **unseen external images** (images not part of the dataset, fetched separately from the web). Impressively, it maintained a high prediction accuracy of around 99% on these new inputs. This demonstrates the robustness and transferability of features learned through the MobileNetV2 backbone.
The success of this model, despite being **trained on a relatively small portion** of the available training data, further underscores the effectiveness of **transfer learning** in image classification tasks.



#### **📁 Dataset**

Kaggle Dataset URL: https://www.kaggle.com/c/dogs-vs-cats



#### **👩‍💻 Author & Contact**

📌 Project by: Nazanin Mahmoudy, 2025                      
📧 Email: Nazaninmahmoudy@gmail.com                       
🔗 GitHub: https://github.com/Nazaninmahmoudi                   
🔗 Kaggle: https://www.kaggle.com/nazaninmahmoudy                     

## Importing Libraries

In [ ]:
import os
import cv2
import urllib.request
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
import random
import cv2
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import roc_curve, auc, precision_recall_curve, average_precision_score, confusion_matrix


###  Dataset Loading & Overview

In [ ]:
dataset_dir = '/kaggle/input/dogs-vs-cats'

In [ ]:
def count_images_in_folder(folder_path):
    valid_extensions = ('.jpg', '.jpeg', '.png')
    return len([f for f in os.listdir(folder_path) if f.lower().endswith(valid_extensions)])

folders = {
    'train/cats': os.path.join(dataset_dir, 'train', 'cats'),
    'train/dogs': os.path.join(dataset_dir, 'train', 'dogs'),
    'test/cats': os.path.join(dataset_dir, 'test', 'cats'),
    'test/dogs': os.path.join(dataset_dir, 'test', 'dogs')
}

for name, path in folders.items():
    count = count_images_in_folder(path)
    print(f"{name}: {count} images")


As we can see, there are 2 Clasess and the number of them is represented 

## Displaying Random Images

In [ ]:
def show_random_image(cat_folder, dog_folder, img_size=224):
   
    selected_class = random.choice(['cat', 'dog'])
    folder = cat_folder if selected_class == 'cat' else dog_folder

    valid_exts = ('.jpg', '.jpeg', '.png')
    image_files = [f for f in os.listdir(folder) if f.lower().endswith(valid_exts)]

    if not image_files:
        print(f"No valid images found in {folder}")
        return

    random_image = random.choice(image_files)
    img_path = os.path.join(folder, random_image)

    img = cv2.imread(img_path)
    if img is None:
        print(f"Couldn't read image {img_path}")
        return

    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img_resized = cv2.resize(img_rgb, (img_size, img_size))

    plt.figure(figsize=(4, 4))
    plt.imshow(img_resized)
    plt.title(f"Random {selected_class.capitalize()} Image")
    plt.axis('off')
    plt.show()


### Load & Preprocess Images

In [ ]:
cat_path = os.path.join(dataset_dir, 'train', 'cats')
dog_path = os.path.join(dataset_dir, 'train', 'dogs')

In [ ]:
show_random_image(cat_path, dog_path)

In [ ]:
show_random_image(cat_path, dog_path)


## Data Preprocessing

 In the following part , we prepare the image data for training our deep learning model. We start by setting two key constants: IMG_SIZE, which defines the target size (224x224 pixels) for each image, and MAX_IMAGES_PER_CLASS, which controls how many images to load from each category.
Deep learning models, especially pretrained architectures like MobileNetV2 that we are using , expect input images to have a fixed shape—typically 224x224 with 3 color channels—so resizing is essential to ensure compatibility.

In [ ]:
IMG_SIZE = 224
MAX_IMAGES_PER_CLASS = 2500

def load_images_from_folder(folder, label, max_images=2500):
    images = []
    labels = []
    count = 0
    for filename in os.listdir(folder):
        if count >= max_images:
            break
        if filename.lower().endswith(('.jpg', '.png', '.jpeg')):
            path = os.path.join(folder, filename)
            img = cv2.imread(path)
            if img is not None:
                img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
                images.append(img)
                labels.append(label)
                count += 1
    return images, labels

In [ ]:
cat_train_folder = os.path.join(dataset_dir, 'train', 'cats')
dog_train_folder = os.path.join(dataset_dir, 'train', 'dogs')
cat_test_folder = os.path.join(dataset_dir, 'test', 'cats')
dog_test_folder = os.path.join(dataset_dir, 'test', 'dogs')


To organize our data, we also define four folder paths: training and testing directories for both cat and dog images. These paths will later be used to load and balance our dataset across the two classes.

In [ ]:
cat_train_images, cat_train_labels = load_images_from_folder(cat_train_folder, 0, MAX_IMAGES_PER_CLASS)
dog_train_images, dog_train_labels = load_images_from_folder(dog_train_folder, 1, MAX_IMAGES_PER_CLASS)

cat_test_images, cat_test_labels = load_images_from_folder(cat_test_folder, 0)
dog_test_images, dog_test_labels = load_images_from_folder(dog_test_folder, 1)

This block loads cat and dog images from the training and test directories. Each image is resized to a consistent size (224x224) for input into the model and labeled as 0 (cat) or 1 (dog). We limit the number of training images per class but load all test images. This ensures balanced training data and a complete, unbiased test set for evaluation.

### Scaling Data

In [ ]:
X_train = np.array(cat_train_images + dog_train_images, dtype=np.float32) / 255.0
Y_train = np.array(cat_train_labels + dog_train_labels)

X_test = np.array(cat_test_images + dog_test_images, dtype=np.float32) / 255.0
Y_test = np.array(cat_test_labels + dog_test_labels)

In this part, we are normalizing the image pixel values from their original range of [0, 255] (standard for 8-bit RGB images) to [0.0, 1.0]

## Splitting Data

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, test_size=0.2, random_state=42, stratify=Y_train)

By keeping the class distribution balanced between training and validation sets, we ensure fair representation of each class.
The validation set lets us track the model’s performance during training while keeping the test set untouched for unbiased final evaluation.


In [ ]:
print(f"Train samples: {X_train.shape[0]}")
print(f"Validation samples: {X_val.shape[0]}")
print(f"Test samples: {X_test.shape[0]}")

### Model Architecture and Transfer Learning Setup

#### MobileNetV2

In [ ]:
base_model = tf.keras.applications.MobileNetV2(
    input_shape=(224, 224, 3),
    include_top=False,
    weights='imagenet'
)

base_model.trainable = False

model = tf.keras.Sequential([
    base_model,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(2, activation='softmax') 
])

The base model is MobileNetV2, a pre-trained convolutional neural network designed for efficient image feature extraction.

By freezing the base model’s weights (trainable=False), we prevent altering its learned features during training, which helps speed up training and avoid overfitting given our smaller dataset.
After the base model, we add a global average pooling layer to reduce the spatial dimensions into a single feature vector per image. 
This is followed by a dense layer with 128 neurons and ReLU activation to learn higher-level features specific to our classification task.
A dropout layer randomly disables 30% of neurons during training, serving as regularization to further prevent overfitting. 
Finally, the output layer with two neurons and softmax activation produces the predicted probabilities for each class.

### Compiling the model

In [ ]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
model.summary()

### Model Summary and Parameters Breakdown

As we can see , The base MobileNetV2 model outputs feature maps of size 7x7 with 1280 channels, containing about 2.25 million parameters. These parameters are non-trainable because we froze the base model’s weights.                       
The global average pooling layer reduces the spatial dimensions to a 1280-length feature vector per image without adding parameters.                            
A fully connected dense layer with 128 neurons adds around 164 thousand trainable parameters, enabling the model to learn task-specific features.                             
the output dense layer with 2 neurons corresponds to our two classes (cats and dogs), with only 258 trainable parameters.

## Trainin the model

In [ ]:
history = model.fit(X_train, Y_train,
                    epochs=5,
                    batch_size=16,
                    validation_data=(X_val, Y_val))

During training, the model’s performance is also evaluated on the validation set after each epoch

## Model Evaluation

In [ ]:
test_loss, test_accuracy = model.evaluate(X_test, Y_test)
print(f"Test Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_accuracy * 100:.2f}%\n")

y_pred_probs = model.predict(X_test)
y_pred = np.argmax(y_pred_probs, axis=1)

print("Classification Report:")
print(classification_report(Y_test, y_pred, target_names=["Cat", "Dog"]))

### Classification Report Anylizing

The classification report demonstrates that the model performs remarkably well on the test set of 5,000 images, achieving an overall accuracy of 97%. Both classes—Cat and Dog—exhibit high and balanced metrics. The Cat class achieved a precision of 0.97 and a recall of 0.98, while the Dog class obtained a precision of 0.98 and a recall of 0.97. These strong values yield F1-scores of 0.97 for both classes, indicating a well-balanced trade off between precision and recall. Additionally, the macro and weighted averages are identical across all metrics, suggesting that the model is not biased toward any specific class and maintains consistent performance throughout.

A key factor contributing to this high performance is the use of **transfer learning**, specifically through a pretrained **MobileNetV2** model. Since MobileNetV2 has been trained on the large and diverse **ImageNet** dataset, it has already learned valuable low-level and mid-level visual features such as edges, textures, and shapes.     
This prior knowledge significantly reduces the amount of data and training time required to achieve good results on a new, smaller dataset.

Moreover, **freezing** the convolutional base while training only the classification head speeds up training and acts as a regularizer, helping to reduce overfitting. This is particularly beneficial when working with limited datasets, as it prevents the model from memorizing the training data.

## Confusion Matrix

In [ ]:
cm = confusion_matrix(Y_test, y_pred)
plt.figure(figsize=(6, 5))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=["Cat", "Dog"], yticklabels=["Cat", "Dog"])
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.title("Confusion Matrix")
plt.show()


As we can see, the confusion matrix demonstrates that the model performs very well in distinguishing between cats and dogs. Out of the total predictions, 2,438 cat images and 2,422 dog images were correctly classified, while only 77 cats were misclassified as dogs and 78 dogs as cats. This indicates that the model has high accuracy and balanced performance across both classes, making it reliable for binary

## Plot Normalized Confusion Matrix and Print Macro/Micro Scores


In [ ]:
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

def plot_normalized_confusion_matrix(y_true, y_pred, class_names):
    cm = confusion_matrix(y_true, y_pred)
    cm_norm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

    plt.figure(figsize=(10, 8))
    ax = sns.heatmap(cm_norm, annot=True, fmt='.2%', cmap='Blues',
                     xticklabels=class_names, yticklabels=class_names)

    if cm.shape == (2, 2):
        tn, fp, fn, tp = cm.ravel()
        labels = np.array([
            [f'TN = {tn}', f'FP = {fp}'],
            [f'FN = {fn}', f'TP = {tp}']
        ])
        for i in range(2):
            for j in range(2):
                ax.text(j + 0.5, i + 0.5, '\n' + labels[i, j],
                        ha='center', va='center', color='black', fontsize=10, weight='bold')

    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.title('Normalized Confusion Matrix with TP, TN, FP, FN')
    plt.show()

    precision_macro = precision_score(y_true, y_pred, average='macro')
    recall_macro = recall_score(y_true, y_pred, average='macro')
    f1_macro = f1_score(y_true, y_pred, average='macro')

    precision_micro = precision_score(y_true, y_pred, average='micro')
    recall_micro = recall_score(y_true, y_pred, average='micro')
    f1_micro = f1_score(y_true, y_pred, average='micro')

    print(f"Macro Precision: {precision_macro:.3f}, Recall: {recall_macro:.3f}, F1-score: {f1_macro:.3f}")
    print(f"Micro Precision: {precision_micro:.3f}, Recall: {recall_micro:.3f}, F1-score: {f1_micro:.3f}")


In [ ]:
class_names = ['Cat', 'Dog']

plot_normalized_confusion_matrix(Y_test, y_pred, class_names)


The evaluation metrics demonstrate a very strong and balanced performance by the model. The macro-averaged precision, recall, and F1-score are all approximately 0.972, indicating that the model performs consistently well across both classes **without bias**.
Similarly, the micro-averaged precision, recall, and F1-score are also around 0.972. This aggregate measure, based on all individual instances, reflects the model’s overall accuracy on the test set.

## Test Dataset Predictions

In [ ]:
def predict_image(model, image_path, class_names=['Cat', 'Dog'], image_size=224):
    if not os.path.exists(image_path):
        print("File not found!")
        return

    img = cv2.imread(image_path)
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    plt.imshow(img_rgb)
    plt.axis('off')
    plt.title("Input Image")
    plt.show()

    img_resized = cv2.resize(img, (image_size, image_size))
    img_scaled = img_resized / 255.0
    img_input = np.expand_dims(img_scaled, axis=0)  

    prediction = model.predict(img_input)[0]
    predicted_class = np.argmax(prediction)
    confidence = prediction[predicted_class] * 100

    print(f" Prediction: {class_names[predicted_class]} ({confidence:.2f}% confidence)")


In [ ]:
image_path = input(" Enter the path to the image: ")
predict_image(model, image_path)

In [ ]:
image_path = input("Enter the path to the image: ")
predict_image(model, image_path)

## Model Predictions on Unseen Images

In [ ]:
 predict_image(model, image_path, class_names=['Cat', 'Dog'], image_size=224):
    if not os.path.exists(image_path):
        print("File not found:", image_path)
        return

    img = cv2.imread(image_path)
    if img is None:
        print("Could not read image:", image_path)
        return

    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    plt.imshow(img_rgb)
    plt.axis('off')
    plt.title(f"Input Image: {os.path.basename(image_path)}")
    plt.show()

    img_resized = cv2.resize(img, (image_size, image_size))
    img_scaled = img_resized / 255.0
    img_input = np.expand_dims(img_scaled, axis=0)  

    prediction = model.predict(img_input)[0]
    predicted_class = np.argmax(prediction)
    confidence = prediction[predicted_class] * 100

    print(f"Prediction: {class_names[predicted_class]} ({confidence:.2f}% confidence)")
    print("-"*40)

urls = [
    "https://rimage.gnst.jp/livejapan.com/public/article/detail/a/00/01/a0001799/img/en/a0001799_parts_5dce6175d72ef.jpg?20200605180827&q=80",
    "https://rimage.gnst.jp/livejapan.com/public/article/detail/a/00/01/a0001799/img/en/a0001799_parts_5d96d0dc280c5.jpg?20200605180827&q=80",
    ""
   
   
]

save_dir = "/kaggle/working/test_images"
os.makedirs(save_dir, exist_ok=True)


for i, url in enumerate(urls):
    ext = url.split('.')[-1].split('?')[0]
    save_path = os.path.join(save_dir, f"image_{i}.{ext}")

    try:
        urllib.request.urlretrieve(url, save_path)
        print(f"Downloaded image {i} to {save_path}")
        predict_image(model, save_path)
    except Exception as e:
        print(f"Failed to download or predict for {url}: {e}")


## 🎯 Conclusion

This project demonstrated the effectiveness of transfer learning using MobileNetV2 for binary image classification. Despite training on a relatively small subset of the dataset, the model achieved impressive results with **98% accuracy on unseen** test data. This highlights the power of pretrained models in achieving high performance with limited training resources.